# Estudo dos dados da base de dados servicosxis.db

Carlos Ramires

inicio: 15/11/2023
termino: 21/11/2023

Busca das distribuições para Entrada de Chamados e  Tecnicos para uma simulação de capacity

Tabelas importantes da servicosxos.db

servicos: Base raw \
serv_campo: base só com os serviços de campo \
qtd_servicos_'unidade': apresenta a distribuição dos serviços por mês de 2020-01 a 2023-07 \
qtd_tec_mes: Mostra a quantidade de técnicos por unidade por mês\
resumo_servicos: Mostra a quantidade por serviços, por unidade, no periodo\
servicos_mensal: Serviços de campo com Ano_Mes\
servicos_mes: Tem o total de serviços por mes, por unidade\
servicos_tec_mes: Relaciona o número de serviços, por serviço, por técnico, por mes\


### Assumption

#### Tempo médio de atendimento de um pedido

Considerando o tempo entre abertura e encerramento do chamado tem-se o tempo total do atendimento.

Em 

servicos_mes > Total de serviços/mes

serv_campo > serviços, causa, designado, duração

Dados espúrios serão descartados da seguinte maneira:




#### Tempo médio de execução de um serviço

Um mês tem 176 horas sendo 22 horas de instrução o que define 154 horas úteis

Considerando um fator de ponderação fp, 154 horas trabalhadas no mês e o número de serviços entregues, temos:

num_instal * fp + num_vt * fp + num_ret * fp + num_mud * fp = 154

Portanto, o fp de cada técico será:

fp =   154 / (num_instal + num_vt + num_ret + num_mud)

ou seja, o tempo médio dispendido para cada serviço deverá ser tm = num_serv * fp

### Número de Equipes disponíveis por mês

Em [serv_campo] identificar equipes válidas por unidade e contar equipes ativas mês a mês




In [2]:
import sqlite3
import pandas as pd

# Carrega a tabela servicos da base servicosxis.db para um dataframe

# Cria a conexão
cnx = sqlite3.connect('servicosxis.db')

# carrega tabela servicos em df
df = pd.read_sql_query("SELECT * FROM serv_campo", cnx)

tecnicos_df = pd.read_sql_query("SELECT * FROM tecnicos_unidade", cnx)

cnx.close()


In [7]:
# criar lista de técnicos

tecnicos = set(tecnicos_df['Tecnicos'])

# filtrar df só com os tecnicos

df_tecnicos = df.loc[df['Usuario_Designado'].isin(tecnicos)]



In [14]:
# Criar a coluna 'Ano_Mes' na df_tecnicos

# converte string data e hora da coluna no formato datatime

df_tecnicos['Abertura_DataHora'] = pd.to_datetime(df['Abertura_DataHora'])
df_tecnicos['Encerramento_DataHora'] = pd.to_datetime(df['Encerramento_DataHora'])
df_tecnicos['Ano_Mes'] = df_tecnicos['Encerramento_DataHora'].dt.to_period('M')



In [18]:

# df_tecnicos['Ano_Mes'] = df_tecnicos['Ano_Mes'].dt.strftime('%Y-%m')

In [17]:
#Criar Tabela na base com os dados de Tecnicos

# 
# conectar a base

cnx = sqlite3.connect('servicosxis.db')

#Criando a tabela da unidade

tabela = ('serv_campo_tec')

df_tecnicos.to_sql(tabela, cnx, if_exists='replace', index=True)

cnx.close()

In [9]:
# limpar serv_campo das informações sem o técnico ativo

# Torna a coluna Tecnicos uma lista
tecnicos_ativos = set(tecnicos_df['Tecnicos'])

# mantem somente os itens que contenham 'tecnicos ativos'
df_ativos = df.loc[df['Usuario_Designado'].isin(tecnicos_ativos)]





In [22]:
# limpar df_ativos das Causas improdutivas

# Torna a coluna Tecnicos uma lista
causas_produtivas = ['(Instalação OK) Instalação Realizada', 
                        '(Equip Retirado) Equipamento Retirado' ,                     
                        '(ConectorNOK) Conector Danificado' ,                          
                        '(Equip Não Retir) Equipamento Não Retirado Cliente Ausente',
                         '(RompimentoCabo) Rompimento de Cabo Drop',
                        '(ConfigEquip) Configuração do Equipamento',                    
                        '(SubstEquip) Substituição de Equipamento' ,               
                        '(Reorientação) Reorientação ao cliente',                         
                        '(AtualPPPOE) Atualização PPPOE' ,                               
                        '(EncontradoOK) Internet Encontrada OK',                                               
                        '(TrocaSenha) Troca de Senha' ,                                   
                        '(InfoWiFi) Informação Sobre Wi-Fi Diferença 2.4 e 5',                                                      
                        '(ConfigWiFi) Configuração do Wi-Fi',                            
                        '(Não Instalado) Instalação Não Realizada Motivos Técnico',       
                        '(EquipDanificado) Equipamento Danificado',                                           
                        '(EquipNHomol) Equipamento Não Homologado',                        
                        '(MudPontoOK) Mudança de Ponto OK',                                                 
                        '(MudançaPontoOK) Mudança de Ponto OK',                           
                        '(EquipDesligado) Equipamento Desligado/Fora da Tomada',           
                        '(CoberturaWiFi) Cobertura do Wi-Fi' ,                                                
                        '(SemAcessoSite) Sem Acesso a Site Especifico',                     
                        '(Mudança Inv) Mudança Inviável Técnicamente'                               
                        '(VisitaRealizad) Visita realizada com sucesso',                    
                        '( Reorientação) Reorientação ao cliente',                          
                        '( AtualPPPOE) Atualização PPPOE',                                  
                        '(Mudança Ilegal) Mudança Ilegal',                                   
                        '(ResetEquip) Reset de Equipamento',                                 
                        '(MudIlegal) Mudança Ilegal',                                                
                        '(Susp Temp Retir) Suspensão Temporária Retirada' ,                  
                        '(Equip Extraviad) Equipamento Extraviado'                          
                     ]
                       

# mantem somente os itens que contenham 'tecnicos ativos'
df_ok = df_ativos.loc[df['Causa'].isin(causas_produtivas)]




In [24]:
topicos = df_ok['Causa'].value_counts()

print(topicos)

(Instalação OK) Instalação Realizada                          23956
(Equip Retirado) Equipamento Retirado                          8155
(ConectorNOK) Conector Danificado                              5328
(Equip Não Retir) Equipamento Não Retirado Cliente Ausente     3285
(RompimentoCabo) Rompimento de Cabo Drop                       3037
(ConfigEquip) Configuração do Equipamento                      2560
(SubstEquip) Substituição de Equipamento                       2535
(Reorientação) Reorientação ao cliente                         2386
(AtualPPPOE) Atualização PPPOE                                 1902
(EncontradoOK) Internet Encontrada OK                          1700
(TrocaSenha) Troca de Senha                                    1278
(InfoWiFi) Informação Sobre Wi-Fi Diferença 2.4 e 5             706
(ConfigWiFi) Configuração do Wi-Fi                              573
(Não Instalado) Instalação Não Realizada Motivos Técnico        379
(EquipDanificado) Equipamento Danificado        

In [25]:
#Criar Tabela na base com os dados 

# 
# conectar a base

cnx = sqlite3.connect('servicosxis.db')

#Criando a tabela da unidade

tabela = ('serv_campo_ok')

df_ok.to_sql(tabela, cnx, if_exists='replace', index=True)

cnx.close()


In [ ]:
import numpy as np

# Suponha que 'dados' seja a sua série de dados
dados = [10, 15, 20, 25, 100, 30, 35, 40, 45]

media = np.mean(dados)
desvio_padrao = np.std(dados)

limite_superior = media + 2 * desvio_padrao
limite_inferior = media - 2 * desvio_padrao

outliers = [x for x in dados if x > limite_superior or x < limite_inferior]

print("Outliers:", outliers)


In [ ]:
import numpy as np

# Suponha que 'dados' seja a sua série de dados
dados = [10, 15, 20, 25, 100, 30, 35, 40, 45]

q1 = np.percentile(dados, 25)
q3 = np.percentile(dados, 75)

iqr = q3 - q1
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr

outliers = [x for x in dados if x > limite_superior or x < limite_inferior]

print("Outliers:", outliers)


In [ ]:
# Criar uma lista com os dados de entrada para cada serviço de cada unidade para 
# encontrar a melhor distribuição que a representa





In [ ]:
# Criar uma tabela com os dados de performance de cada técnico
# relacionando o número de serviços executados por mês com o número de horas trabalhadas
# considerar: 
# 

In [ ]:
import sqlite3
import pandas as pd

# Carrega a tabela servicos da base servicosxis.db para um dataframe

# Cria a conexão
cnx = sqlite3.connect('servicosxis.db')

# carrega tabela servicos em df
df = pd.read_sql_query("SELECT * FROM serv_campo", cnx)

cnx.close()

In [ ]:
""" encontrar uma distribuição estatística que melhor represente uma lista 
de valores usando Python. Uma abordagem comum para fazer isso é ajustar os dados a 
diferentes distribuições de probabilidade e escolher a distribuição que melhor se ajusta
aos seus dados com base em critérios como o coeficiente de determinação (R²), o teste de 
aderência de Kolmogorov-Smirnov ou o critério de informação Akaike (AIC).

Aqui está um exemplo de como fazer isso usando a biblioteca scipy.stats:
"""
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# Suponha que você tenha uma lista de valores 'data'
data = [10.2, 11.5, 12.8, 9.7, 11.2, 14.1, 8.5, 10.3, 13.0, 12.1]

# Ajustar os dados a diferentes distribuições
distributions = [stats.norm, stats.expon, stats.gamma, stats.lognorm]

best_fit = None
best_params = None
best_sse = float('inf')

for distribution in distributions:
    params = distribution.fit(data)
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]
    pdf = distribution.pdf(data, loc=loc, scale=scale, *arg)
    sse = np.sum((pdf - data)**2)
    
    if sse < best_sse:
        best_fit = distribution
        best_params = params
        best_sse = sse

# Agora você tem a melhor distribuição encontrada e seus parâmetros
print("Melhor Distribuição:", best_fit.name)
print("Melhores Parâmetros:", best_params)

# Você pode plotar o histograma dos dados e a PDF da melhor distribuição
plt.hist(data, bins=20, density=True, alpha=0.5, color='b')
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = best_fit.pdf(x, *best_params)
plt.plot(x, p, 'k', linewidth=2)
plt.show()
